In [5]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc
Cosmology: Planck18


In [6]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_parameters.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

Enter the run folder name: test3
Loaded 43 object(s) from /Users/david/Code/msc/runs/test3/sncosmo_parameters.csv


,object_id,chisq,ndof,z,t0,x0,x1,c,t0_err,x0_err,x1_err,c_err
0,ZTF19aailqir,1.939871,7,0.05434,58529.240320,0.000424,0.005314,0.300000,0.526847,0.000020,0.719291,0.061484
1,ZTF19aailqli,86.069495,19,0.05800,58535.586782,0.001352,0.198582,-0.025923,0.240896,0.000089,0.368625,0.047729
2,ZTF19aailteb,16.734382,8,0.07800,58524.495904,0.000725,2.999996,-0.022828,0.836042,0.000038,0.979059,0.053891
3,ZTF19aailuvj,4.449338,14,0.07100,58536.548154,0.000838,0.470822,-0.073357,0.313527,0.000055,0.422183,0.047706
4,ZTF19aajwdqt,16.859893,13,0.08800,58538.834933,0.000714,3.000000,0.074628,0.333026,0.000034,0.171552,0.046395


In [ ]:
C_KM_S = 299792.458  # speed of light in km/s

model = sncosmo.Model(source="salt2")
m_B_list = [] # this is the apparent magnitude at the peak of the light curve
mu_list = [] # this is the distance modulus at the peak of the light curve
d_L_Mpc_list = [] # this is the distance in megaparsecs at the peak of the light curve
H0_list = [] # this is the Hubble constant at the peak of the light curve

for _, row in df.iterrows():
    model.set(z=row["z"], t0=row["t0"], x0=row["x0"], x1=row["x1"], c=row["c"])
    # Peak apparent and absolute B magnitude from the fitted model
    m_B = model.source_peakmag("bessellb", "ab")
    M_B = model.source_peakabsmag("bessellb", "ab")
    mu_obs = m_B - M_B
    m_B_list.append(m_B)
    mu_list.append(mu_obs)
    # d_L in parsecs: mu = 5*log10(d_L/10pc) => d_L = 10^(mu/5 + 1) pc
    d_L_pc = 10 ** (mu_obs / 5.0 + 1.0)
    d_L_Mpc = d_L_pc / 1e6
    d_L_Mpc_list.append(d_L_Mpc)
    # H0 = c*z/d_L (low-z approximation), with c in km/s, d_L in Mpc
    H0_list.append(C_KM_S * row["z"] / d_L_Mpc)

out = df[["object_id", "z", "t0", "x0", "x1", "c"]].copy()
out["m_B"] = np.array(m_B_list)
out["mu"] = np.array(mu_list)
out["d_L_Mpc"] = np.array(d_L_Mpc_list)
out["H0"] = np.array(H0_list)

# Print out all the results from sncosmo for each object
for idx, row in out.iterrows():
    print("\nsncosmo results for object_id:", row["object_id"])
    for col in out.columns:
        print(f"{col}: {row[col]}")

out

AttributeError: 'Model' object has no attribute 'peakmag'

In [4]:
# Save to run folder
out_path = run_folder / "distances.csv"
out.to_csv(out_path, index=False)
print(f"Saved to {out_path}")
display(out)

Saved to /Users/david/Code/msc/runs/test2/distances.csv


,object_id,z,t0,x0,x1,c,m_B,mu,d_L_Mpc,H0
0,ZTF17aabtvsy,0.06922,59867.856566,0.000854,2.999999,0.159425,inf,37.543121,322.570202,67.66
1,ZTF17aabvong,0.03400,60607.468131,0.003010,0.027239,0.096397,inf,35.945028,154.527464,67.66
2,ZTF17aacldgo,0.07200,59898.143649,0.000447,-2.999820,0.060099,inf,37.632797,336.170284,67.66
3,ZTF17aadlxmv,0.06200,58878.408576,0.000929,0.735562,0.157813,inf,37.293011,287.476349,67.66
4,ZTF18aaaonon,0.07141,59707.246684,0.000593,-2.999805,-0.039216,inf,37.614046,333.279884,67.66
...,...,...,...,...,...,...,...,...,...,...
413,ZTF19aamhhey,0.04500,58567.629684,0.002050,1.029095,0.072741,inf,36.570966,206.154697,67.66
414,ZTF19aamhhgu,0.03820,58568.194219,0.001441,-2.493633,0.078730,inf,36.204578,174.146801,67.66
415,ZTF19aamhhyo,0.08000,58568.014166,0.000563,-0.193397,-0.014214,inf,37.873493,375.576735,67.66
416,ZTF19aamhlnl,0.07600,58566.281588,0.000610,-1.378491,-0.041084,inf,37.756174,355.823635,67.66
